In [109]:
# Парсинг топа продаж 3dsky
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from datetime import datetime
import time

# Настройка Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Запуск без интерфейса (опционально)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_service = Service("C:\\Users\\mi_al\\Desktop\\chromedriver-win64\\chromedriver.exe")  # Замените на путь к вашему ChromeDriver

# Инициализация констант и переменных
result = []                                                         # итоговый список
maxpage = 2                                                         # сколько страниц топа грузим
basic_url = "https://3dsky.org/3dmodels?order=sell_rating&page="    # адрес (без номера страницы)
pause_time = 5                                                      # время задержки на загрузку в сек
page_size = 60                                                      # число моделей на странице

# Формируем список моделей
for page in range(maxpage):
    # Инициализация браузера
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    
    url = "https://3dsky.org/3dmodels?order=sell_rating&page=" + str(page + 1)
    driver.get(url)

    # Небольшая пауза для загрузки контента
    time.sleep(pause_time)

    # Парсинг данных
    try:
        # Получаем список моделей с заданной страницы топа
        models = driver.find_elements(By.CSS_SELECTOR, "div.model-title")

        for n, model in enumerate(models, 1):
            # Извлечение данных
            curdate = str(datetime.now().date())
            title = model.find_element(By.TAG_NAME, "a").get_attribute("title") if model.find_elements(By.TAG_NAME, "a") else "No title"
            link = model.find_element(By.TAG_NAME, "a").get_attribute("href") if model.find_elements(By.TAG_NAME, "a") else "No link"
            #print(f"Название: {title}, Ссылка: {link}")
            
            # Формируем заготовку строки: номер строки, текущая дата, название модели, ссылка              
            line_num = page*page_size + n
            result.append([line_num, curdate, title, link])

    finally:
        # Закрытие браузера
        driver.quit()

# В цикле по списку получаем подробную информацию по каждой модели
for model in result:
    # Инициализация браузера
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    
    # Открытие страницы конкретной модели
    url = model[3]
    driver.get(url)
    
    # Небольшая пауза для загрузки контента
    time.sleep(pause_time)
    
    # Парсинг данных
    try:
        category = driver.find_elements(By.CSS_SELECTOR, "body > app-root > app-model > app-base-wide > main > section.container.main-base-container > div.row.white-background > div.col-md-12.model-page-top.ng-tns-c55-0 > div.favourite-and-category.ng-tns-c55-0.ng-star-inserted > div:nth-child(1) > div.category.ng-tns-c55-0 > a > span")
        subcategory = driver.find_elements(By.CSS_SELECTOR, "body > app-root > app-model > app-base-wide > main > section.container.main-base-container > div.row.white-background > div.col-md-12.model-page-top.ng-tns-c55-0 > div.favourite-and-category.ng-tns-c55-0.ng-star-inserted > div:nth-child(1) > div.subcategory.ng-tns-c55-0 > a > span")
        platform = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div.model-info-block.ng-tns-c55-0 > table > tbody > tr:nth-child(1) > td:nth-child(2)")
        renders = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div.model-info-block.ng-tns-c55-0 > table > tbody > tr:nth-child(2) > td:nth-child(2) > div")
        published = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div.model-info-block.ng-tns-c55-0 > div.publication-date.ng-tns-c55-0.ng-star-inserted")
        username = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div:nth-child(1) > div > div > a > div > div > div.model-user-name.ng-tns-c55-0")
        followers = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div:nth-child(1) > div > div > a > div > div > div.model-subscribe-count.ng-tns-c55-0.ng-star-inserted")
        selected = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div:nth-child(1) > div > div > div:nth-child(3) > div.price-block.ng-tns-c55-0.ng-star-inserted > div.bookmarks-block-wrapper-mobile.ng-tns-c55-0.ng-star-inserted > div > div")
        
        # Добавляем полученные данные в строку: категория, подкатегория, платформа, рендер
        if category:
            model.append(category[0].text)
        if subcategory:
            model.append(subcategory[0].text)
        if platform:
            model.append(platform[0].text)
        
        render_text = renders[0].text
        if ('Corona' in render_text) and ('V-Ray' in render_text):
            model.extend(['Corona','V-Ray','None'])
        elif ('Corona' in render_text):
            model.extend(['Corona','None','None'])
        elif ('V-Ray' in render_text):
            model.extend(['None','V-Ray','None'])
        elif ('Standard' in render_text):
            model.extend(['None','None','Standard'])
        else:
            model.extend(['None','None','None'])
        
        if published:
            pub_date = " ".join(str(published[0].text).split(" ")[1:])
            date_object = datetime.strptime(pub_date, "%d %B %Y")
            formatted_date = date_object.strftime("%Y-%m-%d")
            model.append(formatted_date)
        if username:
            model.append(username[0].text)
        if followers:
            model.append(int(followers[0].text.split(" ")[0]))
        if selected:
            selected_text = driver.execute_script("return arguments[0].innerText;", selected[0])
            model.append(int(selected_text.strip()))
    
        print(f'Append: {model}')
       
    finally:
        # Закрытие браузера
        driver.quit()

# Выводим результат для контроля
# print(result)

Append: [1, '2024-11-26', 'Christmas tree with fireplace 5', 'https://3dsky.org/3dmodels/show/novogodniaia_elka_s_kaminom_5', 'Decoration', 'Other decorative objects', '3dsMax 2016 + obj', 'Corona', 'None', 'None', '2024-11-11', 'dofen', '313', '143']
Append: [2, '2024-11-26', 'Benuta Leon Cream Wool Rug', 'https://3dsky.org/3dmodels/show/benuta_leon_cream_wool_rug', 'Decoration', 'Carpets', '3dsMax 2015 + obj', 'Corona', 'None', 'None', '2024-11-03', '3Dmitruk', '335', '187']
Append: [3, '2024-11-26', 'Office chair Ashley', 'https://3dsky.org/3dmodels/show/ofisnoe_kreslo_eshli', 'Furniture', 'Office furniture', '3dsMax 2015 + obj', 'Corona', 'None', 'None', '2024-10-13', 'illidan_', '6', '127']
Append: [4, '2024-11-26', 'Indoor Plant Set 89', 'https://3dsky.org/3dmodels/show/indoor_plant_set_89_23', 'Plants', 'Indoor', '3dsMax 2015 + obj', 'Corona', 'V-Ray', 'None', '2024-04-30', 'mh.designer', '211', '38']
Append: [5, '2024-11-26', 'Boucle Fabric', 'https://3dsky.org/3dmodels/show/bo

In [110]:
# Добавляем результат в конец файла
import pandas as pd
file_path = "output.xlsx"

# Чтение данных
existing_data = pd.read_excel(file_path)

# Преобразование текущих данных DataFrame
new_data_df = pd.DataFrame(result[0:], columns=['N','Curdate','Title','Link','Category','Subcategory','Platform','Corona','V-Ray','Standard','Pubdate','Username','Followers','Selected'])

# Конкатенация и запись результата в Excel
updated_data = pd.concat([existing_data, new_data_df], ignore_index=True)
updated_data.to_excel(file_path, index=False)

# Выводим результат для контроля
print(f"Данные успешно сохранены в {file_path}")


Данные успешно сохранены в output.xlsx


In [98]:
# Песочница для апробации парсинга конкретного элемента
# Настройка Selenium
chrome_options = Options()
chrome_options.add_argument("--headless")  # Запуск без интерфейса (опционально)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_service = Service("C:\\Users\\mi_al\\Desktop\\chromedriver-win64\\chromedriver.exe")  # Замените на путь к вашему ChromeDriver

# Инициализация браузера
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Открытие сайта
url = "https://3dsky.org/3dmodels/show/kerns_tumba_divan_ru"
driver.get(url)

# Небольшая пауза для загрузки контента
time.sleep(5)  # Настройте задержку в зависимости от скорости интернета

# Парсинг данных
try:
    # Пример: Найти все элементы с моделями (настройте селектор под сайт)
    models = driver.find_elements(By.CSS_SELECTOR, "#info-desktop > div.model-info-block.ng-tns-c55-0 > table > tbody > tr:nth-child(2) > td:nth-child(2) > div > div")
    print(models[1].text)
    

   
    #published = str(models[0].text)

finally:
    # Закрытие браузера
    driver.quit()

Corona
